# **Memorability Score Assignment - Machine Learning** - **CA684** 

**Exploration:<br>
    [1]Semantic Feature: Captions using Count Vectorizor<br>
    [2]Semantic Feature: Captions using TfIdfVectorizor<br>
    [3]Video Feature:C3D features<br>
    [4]Video Feature:HMP feature<br>
    [5]Aesthetic feature.<br>
    [6]Combination of Caption and C3D<br>
    [7]Combination of C3D and HMP**
                        

**Importing necessary packages and libraries**

In [670]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os

**Importing the Ground Truth Dataset from CSV file**

**The ground truth data set consists of short term memoribility as well as long term memoribility. 
The annotations field is the number of people who were shown the video and memoribility signifies 
the percentage of them who remember the video**

In [671]:
ground_truth = pd.read_csv('D:/DCU/2nd Semester/Assignments/Machine Learning/My Assignment/ground-truth.csv', header=0)

In [672]:
ground_truth.head(5)

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video3.webm,0.924,34,0.846,13
1,video4.webm,0.923,33,0.667,12
2,video6.webm,0.863,33,0.700,10
3,video8.webm,0.922,33,0.818,11
4,video10.webm,0.950,34,0.900,10


**Correlation between the columns**

In [673]:
ground_truth.corr()

,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
short-term_memorability,1.000000,-0.021178,0.325002,-0.013858
nb_short-term_annotations,-0.021178,1.000000,-0.047093,0.285548
long-term_memorability,0.325002,-0.047093,1.000000,0.036051
nb_long-term_annotations,-0.013858,0.285548,0.036051,1.000000


In [674]:
ground_truth = ground_truth.drop(['nb_short-term_annotations', 'nb_long-term_annotations'], axis=1)

## Caption Feature

**Caption comes under Semantic feature, this provides textual description of the video which is in the form of text**

**Importing the captions from a text file**

In [675]:
video_captions = pd.read_csv('G:/.shortcut-targets-by-id/1UWIdExqb5P94RLqkaiODcwFC2v7g3lB2/CA684_Assignment/Dev-set/Captions/dev-set_video-captions.txt',delimiter='\t',header= None,names=('video','Captions'))

In [676]:
video_captions.head(5)

,video,Captions
0,video3.webm,blonde-woman-is-massaged-tilt-down
1,video4.webm,roulette-table-spinning-with-ball-in-closeup-shot
2,video6.webm,khr-gangsters
3,video8.webm,medical-helicopter-hovers-at-airport
4,video10.webm,couple-relaxing-on-picnic-crane-shot


In [677]:
df = pd.merge(ground_truth, video_captions, on = 'video')

**Cleaning text data of captions using NLP Libraries**

In [678]:
# Importing the NLP Libraries
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [679]:
# import nltk
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |   

True

**Cleaning the data here I have cleaned special charecters and retaining only text data in lowercase text, Removing Stopwords, 
Storing the cleaned text data into a list named captions**

In [680]:
cleaned_captions = []
for i in range(0, 6000):
    text = re.sub('[^a-zA-Z]', ' ', video_captions['Captions'][i])
    text = text.lower()
    text = text.split()
    text = [word for word in text if not word in set(stopwords.words('english'))]
    text = ' '.join(text)
    cleaned_captions.append(text)

In [681]:
cleaned_captions[1]


'roulette table spinning ball closeup shot'

**According to the Rohit Gupta et. al. paper on Linear Models for Video Memorability Prediction Using Visual and 
Semantic Feature, certain words had more impact than others. 
I am trying to give extra weights to such words and sending as a new feature.**

### Caption

**Weigths are assigned in the range between 0 and 0.20**

In [682]:
negavtive_weights = {'mountains':0.01,'sunset':0.02,'trees':0.03,'mountain':0.04,'clouds':0.05,'river':0.06,'aerialof':0.07,'lake':0.08,'forest':0.09,'kenya':0.10,'landscape':0.11,'valley':0.12,'leaves':0.13,'sun':0.14}

**Weigths are assigned in the range between 0.50 and 0.70**

In [683]:
positive_weights = {'woman':0.65,'eating':0.64,'putting':0.63,'lying':0.62,'girl':0.61,'selfie':0.60,'relaxing':0.59,'jellyfish':0.58,'cat':0.57,'super':0.56,'slow':0.55,'super':0.54,'american':0.53,'potrait':0.52,'pregnant':0.51,'couple':0.50}

In [684]:
pos_list = []
neg_list = []
for i in range(0, 6000):
    local_pos_weight = 0
    local_neg_weight = 0
    text = video_captions['Captions'][i]
    text = text.split()
    for word in text:
        if(word in set(positive_weights.keys())):
                pw = local_pos_weight + positive_weights[word]
        if(word in set(negavtive_weights.keys())):
                nw = local_neg_weight + negavtive_weights[word]
    pos_list.append(pw)
    neg_list.append(nw)

In [685]:
weights_df = pd.DataFrame(
    {'positive_weights': pos_list,
     'negative_weights': neg_list
    })

In [686]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

**TFIDF**

In [687]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()

In [688]:
#Using CountVectorizer for feature Extraction
caption_features = cv.fit_transform(cleaned_captions).toarray()

In [689]:
caption_features[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [690]:
caption_features.shape

(6000, 5087)

**Training Caption Modal**

In [691]:
X = caption_features
y = ground_truth.iloc[:, 1:3].values

**Splitting the dataset into the Training set and Test set**

In [692]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=1)

**Spearmann"s correlation coefficient**

In [693]:
def Get_score(Y_pred,Y_true):
    '''Calculate the Spearmann"s correlation coefficient'''
    Y_pred = np.squeeze(Y_pred)
    Y_true = np.squeeze(Y_true)
    if Y_pred.shape != Y_true.shape:
        print('Input shapes don\'t match!')
    else:
        if len(Y_pred.shape) == 1:
            Res = pd.DataFrame({'Y_true':Y_true,'Y_pred':Y_pred})
            score_mat = Res[['Y_true','Y_pred']].corr(method='spearman',min_periods=1)
            print('The Spearman\'s correlation coefficient is: %.3f' % score_mat.iloc[1][0])
        else:
            for ii in range(Y_pred.shape[1]):
                Get_score(Y_pred[:,ii],Y_true[:,ii])

**a) Linear Regression Model on Captions**

In [694]:
from sklearn.linear_model import LinearRegression
LMregressor = LinearRegression()
LMregressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [695]:
y_pred = LMregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.071
The Spearman's correlation coefficient is: 0.002


**b) Decision Tree Regression Model on Captions**

In [696]:
from sklearn.tree import DecisionTreeRegressor
DTregressor = DecisionTreeRegressor()
DTregressor.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [697]:
y_pred = DTregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.266
The Spearman's correlation coefficient is: 0.096


**c) Random Forest Regression model on Captions**

In [501]:
from sklearn.ensemble import RandomForestRegressor
RFregressor = RandomForestRegressor(n_estimators = 100)
RFregressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [502]:
y_pred = RFregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.394
The Spearman's correlation coefficient is: 0.161


# Checking for N estimator is 250

In [700]:
from sklearn.ensemble import RandomForestRegressor
RFregressor = RandomForestRegressor(n_estimators = 200)
RFregressor.fit(X_train, y_train)

KeyboardInterrupt: 

In [699]:
y_pred = RFregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.397
The Spearman's correlation coefficient is: 0.168


### TFIDF instead of count vectorizer

In [701]:
captions_tfidf_features = tf.fit_transform(cleaned_captions).toarray()
X= captions_tfidf_features
y = ground_truth.iloc[:, 1:3].values

In [702]:
#Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=1)

**a) Linear Regression Model on Captions with TFIDF**

In [703]:
from sklearn.linear_model import LinearRegression
LMregressor = LinearRegression()
LMregressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [704]:
y_pred = LMregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.134
The Spearman's correlation coefficient is: 0.041


**b) Decision Tree Regression Model on Captions with TFIDF**

In [705]:
from sklearn.tree import DecisionTreeRegressor
DTregressor = DecisionTreeRegressor()
DTregressor.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [706]:
y_pred = DTregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.290
The Spearman's correlation coefficient is: 0.133


**c) Random Forest Regression model on Captions with TFIDF**

In [509]:
from sklearn.ensemble import RandomForestRegressor
RFregressor = RandomForestRegressor(n_estimators = 150)
RFregressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=150,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [510]:
y_pred = RFregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.401
The Spearman's correlation coefficient is: 0.184


# Random Forest with 200

In [710]:
from sklearn.ensemble import RandomForestRegressor
RFregressor = RandomForestRegressor(n_estimators = 200)
RFregressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=200,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [711]:
y_pred = RFregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.403
The Spearman's correlation coefficient is: 0.182


# Random forest for 250 TFIF 2

In [708]:
from sklearn.ensemble import RandomForestRegressor
RFregressor = RandomForestRegressor(n_estimators = 250)
RFregressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=250,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [709]:
y_pred = RFregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.407
The Spearman's correlation coefficient is: 0.183


# h1 Random Forest for 260

In [714]:
from sklearn.ensemble import RandomForestRegressor
RFregressor = RandomForestRegressor(n_estimators = 260)
RFregressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=260,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [715]:
y_pred = RFregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.407
The Spearman's correlation coefficient is: 0.184


**d) ANN Sequential Model on Captions with TFIDF**

In [511]:
pip install pyprind


Note: you may need to restart the kernel to use updated packages.


In [512]:
from keras import Sequential
from keras import layers
from keras import regularizers
import pyprind
from collections import Counter

In [513]:
captions_tfidf_features.shape

(6000, 5087)

In [514]:
max_len =5087

In [515]:
X = captions_tfidf_features
y = ground_truth.iloc[:, 1:3].values

In [516]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size=0.2)

In [517]:
# Building a sequntial model with 2 layers.
model = Sequential()
model.add(layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001),input_shape=(max_len,)))
model.add(layers.Dropout(0.6))
model.add(layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2,activation='sigmoid'))

# compile the model 
model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])

# training the model 
history = model.fit(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test))

Train on 4800 samples, validate on 1200 samples
Epoch 1/10
4800/4800 [==============================] - 1s 283us/step - loss: 0.0788 - accuracy: 0.6508 - val_loss: 0.0262 - val_accuracy: 0.7267
Epoch 2/10
4800/4800 [==============================] - 1s 206us/step - loss: 0.0214 - accuracy: 0.6935 - val_loss: 0.0164 - val_accuracy: 0.7267
Epoch 3/10
4800/4800 [==============================] - 1s 203us/step - loss: 0.0163 - accuracy: 0.6985 - val_loss: 0.0149 - val_accuracy: 0.7267
Epoch 4/10
4800/4800 [==============================] - 1s 204us/step - loss: 0.0152 - accuracy: 0.6996 - val_loss: 0.0146 - val_accuracy: 0.7267
Epoch 5/10
4800/4800 [==============================] - 1s 200us/step - loss: 0.0149 - accuracy: 0.6990 - val_loss: 0.0144 - val_accuracy: 0.7267
Epoch 6/10
4800/4800 [==============================] - 1s 197us/step - loss: 0.0149 - accuracy: 0.6992 - val_loss: 0.0143 - val_accuracy: 0.7267
Epoch 7/10
4800/4800 [==============================] - 1s 204us/step - loss

In [518]:
y_pred = model.predict(X_test)
Get_score(y_pred, Y_test)

The Spearman's correlation coefficient is: 0.325
The Spearman's correlation coefficient is: 0.203


**Conclusion:**
In Random Forest Regression TFIDF gave better results compared to word count vectorizer.


## C3D Feature

In [519]:
#Reading C3D Features from a file
def read_C3D(fname):
    """Scan vectors from file"""
    with open(fname) as f:
        for line in f:
            C3D =[float(item) for item in line.split()] # convert to float type, using default separator
    return C3D


In [520]:
#Calling the method for fetching C3D features and storing them into a array list using loop
my_numlist= [] # Creating new list to store the C3D features
my_namelist = [] #Creating a new list to store the names of the videos
path = 'C3D/*.txt'
for filename in glob.glob('C3D/*.txt'):
    s= read_C3D(filename)
    my_numlist.append(s)
    my_namelist.append(((filename.split('/')[-1]).split('.')[0])+ '.webm') 
#adding .webm to the name to make it similar to the video name in ground truth

In [521]:
len(my_numlist[0]) #All 101 features have been captured

101

In [522]:
len(my_numlist)

6000

In [523]:
#creating new column as video and assigning the names of the vidoes
c3d = pd.DataFrame(np.array(my_numlist).reshape(6000,101))
c3d["video"] = my_namelist
c3d["video"] = c3d["video"].str.slice(start=4)

In [524]:
c3d.head() 

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,video
0,0.000090,0.000615,0.003436,0.001281,3.551400e-03,3.128800e-04,0.000042,0.000018,0.000157,1.171300e-04,...,6.900100e-04,1.708900e-04,0.000231,6.371500e-04,4.001000e-05,6.142000e-05,0.000075,0.000002,0.001323,video10.webm
1,0.002312,0.009966,0.027439,0.000006,2.155300e-04,1.394600e-04,0.000111,0.000029,0.000046,2.388820e-03,...,1.226390e-03,3.939800e-04,0.008439,1.255300e-04,1.976800e-04,1.048400e-04,0.075047,0.287573,0.003510,video100.webm
2,0.468035,0.489501,0.000079,0.000010,2.000000e-07,1.540000e-06,0.000168,0.000010,0.000034,5.200000e-07,...,1.300000e-07,7.000000e-08,0.000006,1.790000e-06,9.200000e-07,1.950000e-06,0.000127,0.000714,0.000408,video1001.webm
3,0.014036,0.000492,0.000233,0.000028,6.320000e-06,1.330000e-06,0.000006,0.000002,0.000007,4.002000e-05,...,4.673000e-05,1.878000e-05,0.000049,3.068000e-05,1.310000e-06,7.410000e-06,0.830456,0.002719,0.001101,video1003.webm
4,0.046734,0.000868,0.000005,0.000008,2.000000e-08,5.600000e-07,0.000013,0.000004,0.000001,5.990000e-06,...,1.000000e-08,4.500000e-07,0.000004,2.400000e-07,1.100000e-07,3.300000e-07,0.000002,0.000018,0.000080,video1004.webm


**Merging the datsets with ground truth and C3D features using Video column**

In [525]:
c3d_df = pd.merge(ground_truth, c3d, on='video')

In [526]:
c3d_df.head()

,video,short-term_memorability,long-term_memorability,0,1,2,3,4,5,6,...,91,92,93,94,95,96,97,98,99,100
0,video3.webm,0.924,0.846,0.020249,0.001578,0.000826,0.000945,0.000063,0.000003,0.001162,...,0.001042,0.000161,0.000257,0.046617,0.000156,0.000006,0.000537,0.000339,0.008437,0.000470
1,video4.webm,0.923,0.667,0.000118,0.000891,0.000188,0.000045,0.000063,0.000002,0.000641,...,0.000582,0.000393,0.000864,0.000947,0.000136,0.000007,0.000360,0.000159,0.001025,0.000020
2,video6.webm,0.863,0.700,0.011765,0.000746,0.000784,0.000013,0.000007,0.000028,0.000041,...,0.000224,0.000003,0.000031,0.002538,0.000104,0.000005,0.000064,0.005380,0.001027,0.001384
3,video8.webm,0.922,0.818,0.000223,0.000165,0.000007,0.000016,0.000005,0.000014,0.000154,...,0.000046,0.000009,0.000023,0.000053,0.000048,0.000019,0.000001,0.000004,0.000380,0.000029
4,video10.webm,0.950,0.900,0.000090,0.000615,0.003436,0.001281,0.003551,0.000313,0.000042,...,0.000037,0.000690,0.000171,0.000231,0.000637,0.000040,0.000061,0.000075,0.000002,0.001323


In [527]:
c3d_features = c3d_df.iloc[:,3:104].values

In [528]:
c3d_features.shape

(6000, 101)

In [529]:
X = c3d_features
y = ground_truth.iloc[:, 1:3].values

**Splitting the dataset into the Training set and Test set**

In [530]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

**a) Linear Regression Model on C3D**


In [531]:
from sklearn.linear_model import LinearRegression
LMregressor = LinearRegression()
LMregressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [532]:
y_pred = LMregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.238
The Spearman's correlation coefficient is: 0.116


**b) Decision Tree Regression Model on C3D**

In [533]:
from sklearn.tree import DecisionTreeRegressor
DTregressor = DecisionTreeRegressor()
DTregressor.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [534]:
y_pred = DTregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.041
The Spearman's correlation coefficient is: 0.085


**c) Random Forest Regression model on C3D**

In [535]:
from sklearn.ensemble import RandomForestRegressor
RFregressor = RandomForestRegressor(n_estimators=10)
RFregressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [536]:
#Predicting the values using the trained model
y_pred = RFregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.210
The Spearman's correlation coefficient is: 0.118


**d) ANN model on C3D alone**

In [537]:
# !pip install pyprind
# !pip install keras
!pip install --upgrade tensorflow

Requirement already up-to-date: tensorflow in c:\anaconda3\lib\site-packages (2.1.0)


In [538]:
from keras import Sequential
from keras import layers
from keras import regularizers
import pyprind
from collections import Counter

In [539]:
c3d_features.shape


(6000, 101)

In [540]:
max_len =101


In [541]:
X = c3d_features
y = ground_truth.iloc[:, 1:3].values

In [542]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size=0.2,random_state=1)


In [543]:
# Building a sequntial model with 2 layers.
model = Sequential()
model.add(layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001),input_shape=(max_len,)))
model.add(layers.Dropout(0.6))
model.add(layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2,activation='sigmoid'))

# compile the model 
model.compile(optimizer='rmsprop',loss='mse',metrics=['accuracy'])

# training the model 
history = model.fit(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test))

Train on 4800 samples, validate on 1200 samples
Epoch 1/10
4800/4800 [==============================] - 0s 64us/step - loss: 0.0902 - accuracy: 0.6073 - val_loss: 0.0274 - val_accuracy: 0.7233
Epoch 2/10
4800/4800 [==============================] - 0s 33us/step - loss: 0.0206 - accuracy: 0.6979 - val_loss: 0.0147 - val_accuracy: 0.7233
Epoch 3/10
4800/4800 [==============================] - 0s 34us/step - loss: 0.0157 - accuracy: 0.7010 - val_loss: 0.0138 - val_accuracy: 0.7233
Epoch 4/10
4800/4800 [==============================] - 0s 35us/step - loss: 0.0151 - accuracy: 0.7000 - val_loss: 0.0137 - val_accuracy: 0.7233
Epoch 5/10
4800/4800 [==============================] - 0s 35us/step - loss: 0.0149 - accuracy: 0.7002 - val_loss: 0.0136 - val_accuracy: 0.7233
Epoch 6/10
4800/4800 [==============================] - 0s 36us/step - loss: 0.0148 - accuracy: 0.7002 - val_loss: 0.0137 - val_accuracy: 0.7233
Epoch 7/10
4800/4800 [==============================] - 0s 38us/step - loss: 0.014

In [544]:
y_pred = model.predict(X_test)
Get_score(y_pred, Y_test)

The Spearman's correlation coefficient is: 0.299
The Spearman's correlation coefficient is: 0.160


## **Merging** both the caption features and C3D features


In [545]:
#Using Count vector features
c3dAndCaption_features = np.concatenate((caption_features, c3d_features), axis=1)

In [546]:
#Using TFIDF features
c3dAndCaption_features = np.concatenate((captions_tfidf_features, c3d_features), axis=1)

In [547]:
c3dAndCaption_features.shape

(6000, 5188)

In [548]:
X = c3dAndCaption_features
y = ground_truth.iloc[:, 1:3].values

In [549]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20,random_state=1)

**a) Linear Regression Model on Captions and C3D features**

In [550]:
from sklearn.linear_model import LinearRegression
LMregressor = LinearRegression()
LMregressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [551]:
y_pred = LMregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.100
The Spearman's correlation coefficient is: 0.031


**b) Decision Tree Regression Model on Captions and C3D features**

In [552]:
from sklearn.tree import DecisionTreeRegressor
DTregressor = DecisionTreeRegressor()
DTregressor.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [553]:
y_pred = DTregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.117
The Spearman's correlation coefficient is: 0.018


**c) Random Forest Regression model on Captions and C3D features**

In [554]:
from sklearn.ensemble import RandomForestRegressor
RFregressor = RandomForestRegressor(n_estimators = 10)
RFregressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [555]:
y_pred = RFregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.262
The Spearman's correlation coefficient is: 0.094


**d) ANN**

In [556]:
from keras import Sequential
from keras import layers
from keras import regularizers
import pyprind
from collections import Counter

In [557]:
c3dAndCaption_features.shape


(6000, 5188)

In [558]:
max_len =5188


In [559]:
X = c3dAndCaption_features
y = ground_truth.iloc[:, 1:3].values

In [560]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size=0.1,random_state=1)


In [561]:
model = Sequential()
model.add(layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001),input_shape=(max_len,)))
model.add(layers.Dropout(0.6))
model.add(layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2,activation='sigmoid'))

# compile the model 
model.compile(optimizer='rmsprop',loss='mse',metrics=['accuracy'])

# training the model 
history = model.fit(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test))

Train on 5400 samples, validate on 600 samples
Epoch 1/10
5400/5400 [==============================] - 1s 221us/step - loss: 0.0620 - accuracy: 0.6167 - val_loss: 0.0198 - val_accuracy: 0.7200
Epoch 2/10
5400/5400 [==============================] - 1s 189us/step - loss: 0.0178 - accuracy: 0.6956 - val_loss: 0.0151 - val_accuracy: 0.7200
Epoch 3/10
5400/5400 [==============================] - 1s 183us/step - loss: 0.0153 - accuracy: 0.7028 - val_loss: 0.0146 - val_accuracy: 0.7200
Epoch 4/10
5400/5400 [==============================] - 1s 195us/step - loss: 0.0149 - accuracy: 0.7030 - val_loss: 0.0147 - val_accuracy: 0.7200
Epoch 5/10
5400/5400 [==============================] - 1s 185us/step - loss: 0.0148 - accuracy: 0.7031 - val_loss: 0.0146 - val_accuracy: 0.7200
Epoch 6/10
5400/5400 [==============================] - 1s 190us/step - loss: 0.0148 - accuracy: 0.7031 - val_loss: 0.0146 - val_accuracy: 0.7200
Epoch 7/10
5400/5400 [==============================] - 1s 191us/step - loss:

In [562]:
y_pred = model.predict(X_test)
Get_score(y_pred, Y_test)

The Spearman's correlation coefficient is: 0.351
The Spearman's correlation coefficient is: 0.170


## Aesthetic features

**Another Video feature it can be explored is aesthetic feature. These are a collection of features used in the prediction of visual aesthetics, composed of color, texture and object based 
descriptors, aggregated at video level by median and mean methods.**

In [563]:
#function to read aesthetic feature
def aes_fea_read(df, filename):
 df1 = pd.read_csv(filename, header=None)
 df1['video'] = filename[filename.find('video'):filename.find('.txt')] + '.webm'
 df = df.append(df1, ignore_index=True)
 return df

In [564]:
#read file names from the path mentioned
def reading_file_names(path):
 files = []
 file_number=0
 # r=root, d=directories, f = files
 for r, d, f in os.walk(path):
     for file in f:
         if '.txt' in file:
             files.append(os.path.join(r, file))
             file_number = file_number + 1
 print("total files read = " + str(file_number))
 return files

In [565]:
#read aesthetic feature train data
path='D:/DCU/2nd Semester/Assignments/Machine Learning/My Assignment/dev-set_aesthetic-features/features/aesthetic_visual_features/aesthetic_feat_dev-set_median' #(need to chnage the path as per the directory the C3D or any other files are stored in)
files = reading_file_names(path)
dataframe_aes = pd.DataFrame()
for filename in files:
   dataframe_aes = aes_fea_read(dataframe_aes, filename)

total files read = 6022


In [566]:
dataframe_aes.head(5)

,0,1,2,3,4,5,6,7,8,9,...,100,101,102,103,104,105,106,107,108,video
0,0.427258,0.381448,0.582746,0.409475,0.489531,0.198180,39.394310,0.583333,0.039983,7.0,...,0.000065,0.000000,0.442575,0.347355,1.285838,0.0,-0.279630,-0.012414,0.197281,video10.webm
1,0.160579,0.238155,0.408475,0.180416,0.366411,0.345585,47.599246,0.083333,0.002136,2.0,...,-0.000081,0.116705,0.084845,0.145045,1.053873,0.0,0.719529,1.099627,1.690907,video100.webm
2,0.302134,0.330636,0.496383,0.287178,0.430446,0.246384,43.865849,0.083333,0.030235,3.0,...,0.000029,0.016424,0.350831,0.370096,1.415410,0.0,-0.126543,-0.033257,-0.062579,video1001.webm
3,0.289857,0.261362,0.534633,0.291529,0.485201,0.299456,47.997030,0.065891,0.025044,2.0,...,-0.007455,0.864074,0.473810,0.456259,1.940068,0.0,-0.114065,0.232043,0.398709,video1003.webm
4,0.637255,0.319937,0.317086,0.229754,0.261037,0.543062,52.033505,0.584848,0.084615,3.0,...,0.000404,0.183904,0.347324,0.425562,0.474023,0.0,0.292011,0.338289,-0.196339,video1004.webm


In [567]:
dataframe_aes_hdf = pd.merge(ground_truth, dataframe_aes, on='video')

In [568]:
dataframe_aes_hdf.head()

,video,short-term_memorability,long-term_memorability,0,1,2,3,4,5,6,...,99,100,101,102,103,104,105,106,107,108
0,video3.webm,0.924,0.846,0.180874,0.715626,0.198930,0.610453,0.121610,0.692880,55.008914,...,-0.002154,-0.000850,0.000000,0.260845,0.294153,0.575061,0.0,0.129661,0.412381,0.788761
1,video4.webm,0.923,0.667,0.146076,0.550070,0.460729,0.424397,0.341807,0.398212,47.307755,...,0.000007,-0.000399,0.920270,0.337932,0.382134,0.943198,0.0,0.692224,0.762354,-0.561793
2,video6.webm,0.863,0.700,0.063898,0.827439,0.211925,0.720297,0.129499,0.722464,60.535453,...,0.002896,0.002309,0.374684,0.019155,0.187358,0.512601,0.0,0.438261,-0.219560,0.276687
3,video8.webm,0.922,0.818,0.380539,0.134683,0.746891,0.171604,0.697045,0.452995,47.827289,...,0.000568,0.000514,0.970656,0.435550,0.431595,1.142377,0.0,0.424483,0.326030,0.688481
4,video10.webm,0.950,0.900,0.427258,0.381448,0.582746,0.409475,0.489531,0.198180,39.394310,...,0.005307,0.000065,0.000000,0.442575,0.347355,1.285838,0.0,-0.279630,-0.012414,0.197281


In [569]:
aesthetic_features = dataframe_aes_hdf.iloc[:,3:112].values

In [571]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20,random_state=1)

**a) Linear Regression Model on Aesthetic**

In [572]:
from sklearn.linear_model import LinearRegression
LMregressor = LinearRegression()
LMregressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [573]:
y_pred = LMregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.100
The Spearman's correlation coefficient is: 0.031


**b) Decision Tree Regression Model on Aesthetic**

In [574]:
from sklearn.tree import DecisionTreeRegressor
DTregressor = DecisionTreeRegressor()
DTregressor.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [575]:
y_pred = DTregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.125
The Spearman's correlation coefficient is: 0.040


**c) Random Forest Regression model on Aesthetic Features**

In [576]:
from sklearn.ensemble import RandomForestRegressor
RFregressor = RandomForestRegressor()
RFregressor.fit(X_train, y_train)

C:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [577]:
y_pred = RFregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.314
The Spearman's correlation coefficient is: 0.131


**d) ANN Sequential Model on Aesthetic Features**

In [578]:
!pip install pyprind

In [579]:
from keras import Sequential
from keras import layers
from keras import regularizers
import pyprind
from collections import Counter

In [580]:
aesthetic_features.shape

(6000, 109)

In [581]:
max_len = 109

In [582]:
X = aesthetic_features
y = ground_truth.iloc[:, 1:3].values

In [583]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size=0.2)

**Building a sequntial model with 2 layers.**

In [584]:
model = Sequential()
model.add(layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001),input_shape=(max_len,)))
model.add(layers.Dropout(0.6))
model.add(layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2,activation='sigmoid'))

In [585]:
# compile the model 
model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])

In [586]:
# training the model 
history = model.fit(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test))

Train on 4800 samples, validate on 1200 samples
Epoch 1/10
4800/4800 [==============================] - 0s 88us/step - loss: 0.1583 - accuracy: 0.6812 - val_loss: 0.1010 - val_accuracy: 0.6950
Epoch 2/10
4800/4800 [==============================] - 0s 39us/step - loss: 0.0923 - accuracy: 0.7031 - val_loss: 0.0851 - val_accuracy: 0.6950
Epoch 3/10
4800/4800 [==============================] - 0s 41us/step - loss: 0.0820 - accuracy: 0.7056 - val_loss: 0.0787 - val_accuracy: 0.6950
Epoch 4/10
4800/4800 [==============================] - 0s 39us/step - loss: 0.0759 - accuracy: 0.7077 - val_loss: 0.0741 - val_accuracy: 0.6950
Epoch 5/10
4800/4800 [==============================] - 0s 39us/step - loss: 0.0716 - accuracy: 0.7071 - val_loss: 0.0701 - val_accuracy: 0.6950
Epoch 6/10
4800/4800 [==============================] - 0s 39us/step - loss: 0.0678 - accuracy: 0.7067 - val_loss: 0.0670 - val_accuracy: 0.6950
Epoch 7/10
4800/4800 [==============================] - 0s 45us/step - loss: 0.065

In [587]:
y_pred = model.predict(X_test)
Get_score(y_pred, Y_test)

The Spearman's correlation coefficient is: 0.025
The Spearman's correlation coefficient is: 0.033


## HMP Feature

**Defining the method for reading HMP Features from a file**

In [588]:
def read_HMP(fname):
    """Scan HMP(Histogram of Motion Patterns) features from file"""
    with open(fname) as f:
        for line in f:
            pairs=line.split()
            HMP_temp = { int(p.split(':')[0]) : float(p.split(':')[1]) for p in pairs}
    # there are 6075 bins, fill zeros
    HMP = np.zeros(6075)
    for idx in HMP_temp.keys():
        HMP[idx-1] = HMP_temp[idx]            
    return HMP

In [589]:
HMP_feature_list= []
video_names_list = []
path = 'HMP/*.txt'
for filename in glob.glob('HMP/*.txt'):
    name = ((filename.split('/')[-1]).split('.')[0]+'.webm')
    video_names_list.append(name)
    HMP_features = read_HMP(filename)
    HMP_feature_list.append(HMP_features)

In [590]:
HMP_features = pd.DataFrame(np.array(HMP_feature_list).reshape(6000,6075))
HMP_features["video"] = video_names_list
HMP_features["video"] = HMP_features["video"].str.slice(start=4)

In [591]:
HMP_features.head()

,0,1,2,3,4,5,6,7,8,9,...,6066,6067,6068,6069,6070,6071,6072,6073,6074,video
0,0.005026,0.001356,0.000055,0.0,0.000665,0.000029,0.0,0.000000,0.000024,0.0,...,0.000882,0.000200,0.000009,0.000559,0.001097,0.000018,0.000632,0.001128,0.000064,video10.webm
1,0.019473,0.005004,0.000098,0.0,0.001991,0.000038,0.0,0.000000,0.000011,0.0,...,0.000056,0.000013,0.000000,0.000045,0.000178,0.000022,0.000071,0.000256,0.000085,video100.webm
2,0.007250,0.002722,0.000050,0.0,0.001070,0.000055,0.0,0.000000,0.000035,0.0,...,0.000090,0.000060,0.000000,0.000100,0.000234,0.000010,0.000164,0.000413,0.000025,video1001.webm
3,0.071224,0.011862,0.000320,0.0,0.007180,0.000301,0.0,0.000002,0.000208,0.0,...,0.000093,0.000049,0.000000,0.000086,0.000208,0.000019,0.000063,0.000236,0.000139,video1003.webm
4,0.053318,0.009491,0.000454,0.0,0.004754,0.000414,0.0,0.000004,0.000155,0.0,...,0.000290,0.000113,0.000007,0.000190,0.000808,0.000111,0.000317,0.000974,0.001045,video1004.webm


In [592]:
hdf = pd.merge(ground_truth, HMP_features , on='video')

In [593]:
hdf.head()

,video,short-term_memorability,long-term_memorability,0,1,2,3,4,5,6,...,6065,6066,6067,6068,6069,6070,6071,6072,6073,6074
0,video3.webm,0.924,0.846,0.125563,0.024036,0.000314,0.0,0.015864,0.000358,0.0,...,0.000000,0.000393,0.000279,0.000000,0.000289,0.001926,0.000000,0.000086,0.000580,0.000000
1,video4.webm,0.923,0.667,0.007526,0.001421,0.000068,0.0,0.001184,0.000143,0.0,...,0.000053,0.000244,0.000066,0.000000,0.000081,0.000617,0.000094,0.000220,0.000762,0.001224
2,video6.webm,0.863,0.700,0.109584,0.018978,0.000289,0.0,0.008774,0.000208,0.0,...,0.000007,0.000054,0.000045,0.000000,0.000028,0.000291,0.000033,0.000052,0.000258,0.000215
3,video8.webm,0.922,0.818,0.120431,0.013561,0.000277,0.0,0.018974,0.000913,0.0,...,0.000059,0.001110,0.000075,0.000008,0.000333,0.000793,0.000101,0.000588,0.000503,0.000452
4,video10.webm,0.950,0.900,0.005026,0.001356,0.000055,0.0,0.000665,0.000029,0.0,...,0.000009,0.000882,0.000200,0.000009,0.000559,0.001097,0.000018,0.000632,0.001128,0.000064


In [594]:
hmp_features = hdf.iloc[:,3:6078].values

In [595]:
X = hmp_features
Y = ground_truth.iloc[:, 1:3].values

In [597]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20,random_state=1)

**a) Linear Regression Model on HMP**

In [598]:
from sklearn.linear_model import LinearRegression
LMregressor = LinearRegression()
LMregressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [599]:
y_pred = LMregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.025
The Spearman's correlation coefficient is: -0.003


**b) Decision Tree Regression Model on HMP**

In [600]:
from sklearn.tree import DecisionTreeRegressor
DTregressor = DecisionTreeRegressor()
DTregressor.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [601]:
y_pred = DTregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.092
The Spearman's correlation coefficient is: 0.041


**c) Random Forest Regression model on HMPfeatures**



In [602]:
from sklearn.ensemble import RandomForestRegressor
RFregressor = RandomForestRegressor()
RFregressor.fit(X_train, y_train)

C:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [603]:
y_pred = RFregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.221
The Spearman's correlation coefficient is: 0.048


**d) ANN Sequential Model on HMP Features**

In [604]:
!pip install pyprind

In [605]:
from keras import Sequential
from keras import layers
from keras import regularizers
import pyprind
from collections import Counter

In [606]:
hmp_features.shape

(6000, 6075)

In [607]:
max_len = 6075

In [608]:
X = hmp_features
y = ground_truth.iloc[:, 1:3].values

In [610]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size=0.2,random_state=1)

**Building a sequntial model with 2 layers.**

In [611]:
model = Sequential()
model.add(layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001),input_shape=(max_len,)))
model.add(layers.Dropout(0.6))
model.add(layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2,activation='sigmoid'))

# compile the model 
model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])

# training the model 
history = model.fit(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test))

Train on 4800 samples, validate on 1200 samples
Epoch 1/10
4800/4800 [==============================] - 1s 269us/step - loss: 0.0819 - accuracy: 0.6060 - val_loss: 0.0247 - val_accuracy: 0.7233
Epoch 2/10
4800/4800 [==============================] - 1s 232us/step - loss: 0.0215 - accuracy: 0.6967 - val_loss: 0.0154 - val_accuracy: 0.7233
Epoch 3/10
4800/4800 [==============================] - 1s 232us/step - loss: 0.0166 - accuracy: 0.7000 - val_loss: 0.0140 - val_accuracy: 0.7233
Epoch 4/10
4800/4800 [==============================] - 1s 229us/step - loss: 0.0154 - accuracy: 0.6983 - val_loss: 0.0137 - val_accuracy: 0.7233
Epoch 5/10
4800/4800 [==============================] - 1s 230us/step - loss: 0.0152 - accuracy: 0.6998 - val_loss: 0.0136 - val_accuracy: 0.7233
Epoch 6/10
4800/4800 [==============================] - 1s 231us/step - loss: 0.0150 - accuracy: 0.7002 - val_loss: 0.0137 - val_accuracy: 0.7233
Epoch 7/10
4800/4800 [==============================] - 1s 230us/step - loss

In [612]:
y_pred = model.predict(X_test)
Get_score(y_pred, Y_test)

The Spearman's correlation coefficient is: 0.084
The Spearman's correlation coefficient is: 0.066


## Merging HMP and C3d features

In [613]:
#Concatinating c3d features and HMP features
c3dAndHMP_features1 = np.concatenate((c3d_features,hmp_features), axis=1)

In [614]:
#  Inputting the values for X and Y
X = c3dAndHMP_features1
y = ground_truth.iloc[:, 1:3].values

In [615]:
#splitting the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20,random_state=1)

**a) Linear Regression Model on C3D and HMP features**

In [616]:
from sklearn.linear_model import LinearRegression
LMregressor = LinearRegression()
LMregressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [617]:
y_pred = LMregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.052
The Spearman's correlation coefficient is: -0.007


**b) Decision Tree Regression Model on Captions and C3D features**


In [618]:
from sklearn.tree import DecisionTreeRegressor
DTregressor = DecisionTreeRegressor()
DTregressor.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [619]:
y_pred = DTregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.117
The Spearman's correlation coefficient is: 0.064


**c) Random Forest Regression model on C3D and HMP features**

In [620]:
from sklearn.ensemble import RandomForestRegressor
RFregressor = RandomForestRegressor()
RFregressor.fit(X_train, y_train)

C:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [621]:
y_pred = RFregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.258
The Spearman's correlation coefficient is: 0.063


**d) ANN Sequential Modal**

In [622]:
!pip install pyprind

In [623]:
from keras import Sequential
from keras import layers
from keras import regularizers
import pyprind
from collections import Counter

In [624]:
c3dAndHMP_features1.shape

(6000, 6176)

In [625]:
max_len = 6176

In [626]:
X = c3dAndHMP_features1
y = ground_truth.iloc[:, 1:3].values

In [627]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size=0.1)

**Building a sequntial model with 2 layers.**

In [628]:
model = Sequential()
model.add(layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001),input_shape=(max_len,)))
model.add(layers.Dropout(0.6))
model.add(layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2,activation='sigmoid'))

# compile the model 
model.compile(optimizer='rmsprop',loss='mse',metrics=['accuracy'])

# training the model 
history = model.fit(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test))

Train on 5400 samples, validate on 600 samples
Epoch 1/10
5400/5400 [==============================] - 4s 770us/step - loss: 0.0611 - accuracy: 0.6180 - val_loss: 0.0202 - val_accuracy: 0.7183
Epoch 2/10
5400/5400 [==============================] - 4s 700us/step - loss: 0.0176 - accuracy: 0.7019 - val_loss: 0.0155 - val_accuracy: 0.7183
Epoch 3/10
5400/5400 [==============================] - 4s 717us/step - loss: 0.0154 - accuracy: 0.7030 - val_loss: 0.0151 - val_accuracy: 0.7183
Epoch 4/10
5400/5400 [==============================] - 4s 710us/step - loss: 0.0149 - accuracy: 0.7033 - val_loss: 0.0150 - val_accuracy: 0.7183
Epoch 5/10
5400/5400 [==============================] - 4s 711us/step - loss: 0.0147 - accuracy: 0.7033 - val_loss: 0.0150 - val_accuracy: 0.7183
Epoch 6/10
5400/5400 [==============================] - 4s 720us/step - loss: 0.0146 - accuracy: 0.7033 - val_loss: 0.0150 - val_accuracy: 0.7183
Epoch 7/10
5400/5400 [==============================] - 4s 705us/step - loss:

In [629]:
y_pred = model.predict(X_test)
Get_score(y_pred, Y_test)

The Spearman's correlation coefficient is: 0.233
The Spearman's correlation coefficient is: 0.095


**Conclusion:
In my exploration Caption feature with TFidf vectorizor with Random Forest Regression model outperformed all other model and the features
So my final model will be based on TFIDF of captions using Random Forest Regressor**

## Final Model

**Importing the Ground Truth Dataset provided for training from CSV file**

In [630]:
ground_truth_train = pd.read_csv('ground-truth.csv')

In [631]:
ground_truth_train.head()

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video3.webm,0.924,34,0.846,13
1,video4.webm,0.923,33,0.667,12
2,video6.webm,0.863,33,0.700,10
3,video8.webm,0.922,33,0.818,11
4,video10.webm,0.950,34,0.900,10


In [632]:
ground_truth_train = ground_truth_train.drop(['nb_short-term_annotations', 'nb_long-term_annotations'], axis=1)

In [633]:
ground_truth_train.head()

,video,short-term_memorability,long-term_memorability
0,video3.webm,0.924,0.846
1,video4.webm,0.923,0.667
2,video6.webm,0.863,0.700
3,video8.webm,0.922,0.818
4,video10.webm,0.950,0.900


**Importing the Ground Truth for test data from CSV file**

In [634]:
ground_truth_test = pd.read_csv('Test-set/Ground-truth_test/ground_truth_template.csv')

In [635]:
ground_truth_test.head()

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,7494,NaN,33,NaN,12
1,7495,NaN,34,NaN,10
2,7496,NaN,32,NaN,13
3,7497,NaN,33,NaN,10
4,7498,NaN,33,NaN,10


**Importing the training set video caption files**

In [636]:
video_captions_train = pd.read_csv('Dev-set/Captions/dev-set_video-captions.txt',delimiter='\t',header= None,names=('video','Captions'))

In [637]:
video_captions_train.head()

,video,Captions
0,video3.webm,blonde-woman-is-massaged-tilt-down
1,video4.webm,roulette-table-spinning-with-ball-in-closeup-shot
2,video6.webm,khr-gangsters
3,video8.webm,medical-helicopter-hovers-at-airport
4,video10.webm,couple-relaxing-on-picnic-crane-shot


**Importing the test set video caption files**

In [638]:
video_captions_test = pd.read_csv('Test-set/Captions_test/test-set-1_video-captions.txt',delimiter='\t',header= None,names=('video','Captions'))

In [639]:
video_captions_test.head()

,video,Captions
0,video7494.webm,green-jeep-struggling-to-drive-over-huge-rocks
1,video7495.webm,hiking-woman-tourist-is-walking-forward-in-mou...
2,video7496.webm,close-up-of-african-american-doctors-hands-usi...
3,video7497.webm,slow-motion-of-a-man-using-treadmill-in-the-gy...
4,video7498.webm,slow-motion-of-photographer-in-national-park


**We need to extract the features using both the train and test sets so that we can train our models with similar dimensions.**

In [640]:
video_captions = pd.concat([video_captions_train, video_captions_test],ignore_index=True)

In [641]:
video_captions.head()

,video,Captions
0,video3.webm,blonde-woman-is-massaged-tilt-down
1,video4.webm,roulette-table-spinning-with-ball-in-closeup-shot
2,video6.webm,khr-gangsters
3,video8.webm,medical-helicopter-hovers-at-airport
4,video10.webm,couple-relaxing-on-picnic-crane-shot


In [642]:
video_captions.tail()

,video,Captions
7995,video10004.webm,astronaut-in-outer-space-against-the-backdrop-...
7996,video10005.webm,young-women-lying-on-sunbed-and-applying-sun-c...
7997,video10006.webm,doctor-talking-to-patient-using-a-tablet-to-ex...
7998,video10007.webm,businessman-sitting-on-the-beach-on-inflatable...
7999,video10008.webm,woman-eating-ice-cream-and-sitting-in-the-stre...


**Cleaning the text data of Captions using NLP Libraries**

**Importing the NLP Libraries**

In [643]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [644]:
negavtive_weights = {'mountains':0.01,'sunset':0.02,'trees':0.03,'mountain':0.04,'clouds':0.05,'river':0.06,'aerialof':0.07,'lake':0.08,'forest':0.09,'kenya':0.10,'landscape':0.11,'valley':0.12,'leaves':0.13,'sun':0.14}

**Giving higher values to positive coefficient words ( weights chose in range between 0.50 and 0.70)**

In [645]:
positive_weights = {'woman':0.65,'eating':0.64,'putting':0.63,'lying':0.62,'girl':0.61,'selfie':0.60,'relaxing':0.59,'jellyfish':0.58,'cat':0.57,'super':0.56,'slow':0.55,'super':0.54,'american':0.53,'potrait':0.52,'pregnant':0.51,'couple':0.50}

**Removing the special charecters and retaining only text data in lowercase text, Removing Stopwords, Storing the cleaned text data into a list named captions Assigning weights accordingly if certain words exists in the captions**

In [646]:
cleaned_captions = []
pos_list = []
neg_list = []
for i in range(0, 8000):
    pw = 0
    nw = 0
    text = re.sub('[^a-zA-Z]', ' ', video_captions['Captions'][i])
    text = text.lower()
    text = text.split()
    text = [word for word in text if not word in set(stopwords.words('english'))]
    for word in text:
        if(word in set(positive_weights.keys())):
                pw = pw + positive_weights[word]
        if(word in set(negavtive_weights.keys())):
                nw = nw + negavtive_weights[word]
    pos_list.append(pw)
    neg_list.append(nw)
    text = ' '.join(text)
    cleaned_captions.append(text)

In [647]:
weights_df = pd.DataFrame(
    {'positive_weights': pos_list,
     'negative_weights': neg_list
    })

In [648]:
weights_df.shape

(8000, 2)

In [649]:
len(cleaned_captions)

8000

**Extracting the TFIDF score features from the cleaned captions**

In [650]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()

In [651]:
caption_features = tf.fit_transform(cleaned_captions).toarray()

In [652]:
caption_features.shape

(8000, 5762)

In [653]:
CaptionAndWeight_features = np.concatenate((caption_features, weights_df), axis=1)

In [654]:
caption_features_train = CaptionAndWeight_features[:6000]

In [655]:
caption_features_test = CaptionAndWeight_features[6000:]

In [656]:
caption_features_train.shape

(6000, 5764)

In [657]:
caption_features_test.shape

(2000, 5764)

In [658]:
def Get_score(Y_pred,Y_true):
    '''Calculate the Spearmann"s correlation coefficient'''
    Y_pred = np.squeeze(Y_pred)
    Y_true = np.squeeze(Y_true)
    if Y_pred.shape != Y_true.shape:
        print('Input shapes don\'t match!')
    else:
        if len(Y_pred.shape) == 1:
            Res = pd.DataFrame({'Y_true':Y_true,'Y_pred':Y_pred})
            score_mat = Res[['Y_true','Y_pred']].corr(method='spearman',min_periods=1)
            print('The Spearman\'s correlation coefficient is: %.3f' % score_mat.iloc[1][0])
        else:
            for ii in range(Y_pred.shape[1]):
                Get_score(Y_pred[:,ii],Y_true[:,ii])

**Passing these parameters for predicting shortterm memorability only**

In [659]:
X= caption_features_train
y = ground_truth_train.iloc[:, 1:3].values

**Splitting the dataset into the Training set and Validation set**

In [660]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=1)

**Fitting Random Forest Regression model on Captions with TFIDF**

In [661]:
from sklearn.ensemble import RandomForestRegressor
RFregressor = RandomForestRegressor(n_estimators = 500)
RFregressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=500,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [663]:
y_pred = RFregressor.predict(X_test)

In [664]:
finalresults = np.asarray(y_pred)
np.savetxt("finalresults.csv", finalresults, delimiter = ",", fmt="%f")